# Robot Aspirador SLAM (con acceso garantizado a estación de carga)
Esta versión garantiza que siempre exista un camino accesible entre la posición inicial del robot y la estación de carga, incluso con obstáculos generados aleatoriamente.

- 🤖 Robot
- 🔋 Estación de carga
- 🟫 Suciedad
- ⬛ Obstáculo (nunca bloquea totalmente el acceso)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import random
import time
from queue import Queue
from IPython.display import clear_output

#Cargamos las imágenes
img_robot = plt.imread("roomba.png")
img_cargador = plt.imread("base_roomba.png")



def camino_disponible(grid, obstaculos, inicio, objetivo):
    q = Queue()                 # Se crea una cola (Queue) para hacer una búsqueda en anchura (BFS).
    q.put(inicio)               # Se añade la posición de inicio como primer nodo.
    visitados = set()           # Se usa visitados para evitar comprobar celdas repetidas.

    while not q.empty():        # Mientras haya posiciones por explorar…
        actual = q.get()        # Se toma una posición actual de la cola.
        if actual == objetivo:  # Si hemos llegado al destino, hay camino disponible → devolvemos True.
            return True

        for dy, dx in [(-1,0),(1,0),(0,-1),(0,1)]:  # Se exploran los 4 vecinos ortogonales de la celda actual.
            ny, nx = actual[0]+dy, actual[1]+dx
            if 0 <= ny < grid.shape[0] and 0 <= nx < grid.shape[1]: # Verificamos que no se sale del mapa.
                if obstaculos[ny, nx] == 0 and (ny, nx) not in visitados: # Si no es un obstáculo y no se ha visitado antes: Se marca como visitado.
                    visitados.add((ny, nx))                               # Se marca como visitado.
                    q.put((ny, nx))                                       # Se añade a la cola para seguir explorando.
    return False                # Si terminamos de recorrer y no se encuentra un camino, se devuelve False.





class EntornoSLAM:

    def __init__(self, tamaño=None):
        while True:                                             # Se repite la generación hasta que haya conexión entre inicio y estación.
            self.size = tamaño or random.choice([5, 6, 7])      # Se define el tamaño del mapa (5x5, 6x6 o 7x7 aleatoriamente si no se pasa como parámetro).
            self.grid = np.random.choice([0, 1], size=(self.size, self.size), p=[0.3, 0.7])           # Se genera la cuadrícula de suciedad:  0 = limpio   1 = sucio    70% del mapa empieza sucio.
            self.obstaculos = np.random.choice([0, 1], size=(self.size, self.size), p=[0.85, 0.15])   # Se crean obstáculos con un 15% de probabilidad.
            self.obstaculos[0, 0] = 0                           # Se asegura que la posición inicial y la estación de carga nunca tengan obstáculos.
            self.obstaculos[-1, -1] = 0

            if camino_disponible(self.grid, self.obstaculos, (0,0), (self.size-1,self.size-1)):       # Solo se acepta el entorno si hay un camino posible desde el inicio a la estación.
                break

        self.robot_pos = (0, 0)                                  # Se establecen las posiciones iniciales del robot y la estación de carga.
        self.carga_pos = (self.size-1, self.size-1)





    def mostrar(self):
        clear_output(wait=True)                                 # Borra el dibujo anterior para crear una animación fluida.
        fig, ax = plt.subplots(figsize=(self.size, self.size))  # Se crea una figura cuadrada proporcional al tamaño del mapa.

        for i in range(self.size):                              # Se recorre la cuadrícula.
            for j in range(self.size):                          # y_inv invierte las filas para que el origen esté abajo (más natural).
                y_inv = self.size - 1 - i

                if (i, j) == self.robot_pos:                   # Si la celda es del robot o la estación, se insertan sus imágenes correspondientes.
                    ax.imshow(img_robot, extent=(j, j+1, y_inv, y_inv+1), zorder=2)

                elif (i, j) == self.carga_pos:
                    ax.imshow(img_cargador, extent=(j, j+1, y_inv, y_inv+1), zorder=1)

                elif self.obstaculos[i, j] == 1:               # Si es obstáculo → celda negra.
                    ax.add_patch(patches.Rectangle((j, y_inv), 1, 1, color='black'))

                elif self.grid[i, j] == 1:                     # Si es sucia → marrón.
                    ax.add_patch(patches.Rectangle((j, y_inv), 1, 1, color='saddlebrown'))

                else:
                    ax.add_patch(patches.Rectangle((j, y_inv), 1, 1, edgecolor='gray', facecolor='white'))  # Si está limpia → blanca.

        ax.set_xlim(0, self.size)       # Ajusta los límites y oculta los ejes para que se vea como un tablero limpio.
        ax.set_ylim(0, self.size)
        ax.axis('off')
        plt.show()                      # Muestra la figura y espera un poco para que dé sensación de animación.
        time.sleep(0.5)

In [ ]:
#Se define la función con dos parámetros:
# pos: una tupla (y, x) que representa la posición actual del robot.
# entorno: el objeto que representa el entorno del robot (el mapa).

def vecinos_validos(pos, entorno):
    y, x = pos                                    # Se descompone la posición en coordenadas y (fila) y x (columna).
    for dy, dx in [(-1,0),(1,0),(0,-1),(0,1)]:    # Este bucle recorre los 4 movimientos posibles: Arriba (-1, 0) Abajo (+1, 0) Izquierda (0, -1) Derecha (0, +1) (no podríamos realizar movimientos diagonales)
        ny, nx = y+dy, x+dx                       # Calcula la nueva posición candidata (ny, nx) sumando el desplazamiento al valor original.
        if 0 <= ny < entorno.size and 0 <= nx < entorno.size: # Verifica que la nueva posición esté dentro del mapa. Evita errores como acceder a una celda fuera de rango.
            if entorno.obstaculos[ny, nx] == 0:               # Comprueba que la nueva celda no sea un obstáculo. Si es un 1, es un obstáculo → no se puede pasar. Si es un 0, es transitable → se puede usar.
                yield (ny, nx)                                # Devuelve la celda vecina válida. Se usa yield en lugar de return para convertir la función en un generador. Así se puede usar en bucles de forma eficiente, sin construir una lista completa.





def buscar_camino(entorno, inicio, objetivo):
    queue = Queue()                              # Se crea una cola (estructura FIFO).
    queue.put((inicio, []))                      # Se añade el primer nodo: la posición inicial y el camino recorrido hasta ahora ([] porque aún no ha avanzado).
    visitados = set()                            # Un conjunto que guarda las celdas ya exploradas, para no repetir y evitar bucles infinitos.

    while not queue.empty():                     # Mientras haya nodos por explorar...
        actual, camino = queue.get()             # Se saca el siguiente nodo (actual) y su camino asociado hasta este punto.
        if actual == objetivo:                   # Si ya hemos llegado al objetivo, devolvemos el camino completo.
            return camino + [actual]

        for vecino in vecinos_validos(actual, entorno):       # Recorremos todos los vecinos válidos (sin obstáculos y dentro del mapa).
            if vecino not in visitados:                       # Si no hemos visitado ese vecino antes:
                visitados.add(vecino)                         # Lo añadimos a visitados.
                queue.put((vecino, camino + [actual]))        # Lo metemos en la cola junto al camino actualizado.
    return []                                                 # Si se acaba la cola y no hay forma de llegar, se devuelve una lista vacía (no hay camino).





def ejecutar_robot(entorno):
    pendientes = set((i,j) for i in range(entorno.size) for j in range(entorno.size)
                     if entorno.grid[i,j] == 1 and entorno.obstaculos[i,j] == 0)        # Crea un conjunto con todas las celdas sucias (1) que no tienen obstáculos (0).

    while pendientes:                                                                   # Mientras queden casillas por limpiar…
        objetivos = sorted(pendientes, key=lambda pos: abs(pos[0]-entorno.robot_pos[0]) + abs(pos[1]-entorno.robot_pos[1]))         # Se ordenan las celdas sucias por distancia Manhattan al robot (más cerca primero).
        destino = objetivos[0]                                                                                                      # Se selecciona el destino más cercano.
        camino = buscar_camino(entorno, entorno.robot_pos, destino)                                                                 # Se calcula el camino más corto del robot hasta ese destino.

        for paso in camino[1:]:                               # Se mueve paso a paso siguiendo el camino. camino[1:] omite la primera celda (donde ya está el robot).
            entorno.robot_pos = paso                              # Se actualiza la posición del robot.
            entorno.mostrar()                                     # Se llama a mostrar() para visualizar el movimiento.
        entorno.grid[destino] = 0                             # Marca la celda como limpia (0).
        pendientes.remove(destino)                            # La elimina del conjunto de celdas pendientes.
        entorno.mostrar()                                                               # Actualiza la visualización.

    camino = buscar_camino(entorno, entorno.robot_pos, entorno.carga_pos)               # Una vez todo está limpio:

    for paso in camino[1:]:                                                             # Se busca el camino al cargador. Se mueve hasta allí mostrando cada paso.
        entorno.robot_pos = paso
        entorno.mostrar()

    print('✅ Limpieza completada. El robot ha llegado a la estación de carga.')

In [ ]:
entorno = EntornoSLAM()
entorno.mostrar()
ejecutar_robot(entorno)